<a href="https://colab.research.google.com/github/Fatih120/GuildedChatExporter/blob/main/guildedchatexporter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GuildedChatExporter

Export your stuff before it's gone.

## Heavy WIP. Works on channels with text, so text channels, even voice and streaming channels.

1.   Set up a bot within the server you want to export from and create an Authentication Key for it. Paste it in the first box below.
2.   Grab the Channel ID of the channel you want to archive. Have Settings > Advanced > Developer mode enabled, then right click a channel and copy the ID: or, find the ID in the channel URL after /channel/*. Paste it below.
3.   Run the cells one by one, skipping the optional ones since you likely don't need them. Do this by clicking the play circle on the top left. Keep your Google Drive open to see your files.

Need help?
https://github.com/Fatih120/GuildedChatExporter/tree/main


[Mountain of Fatih Guilded](https://guilded.gg/MoF)

[Discord :(](https://discord.com/invite/Cy27FNfQtc)



In [ ]:
# @title Basic setup
API_KEY = "gapi_KEY==" # @param {type:"string"}
SERVER_ID = "" # @param {type:"string"}
CHANNEL_ID = "XXXXXXXX-YYYY-ZZZZ-WWWW-YYYYYYYYYYYY " # @param {type:"string"}
SAVE_DIRECTORY = "/guildedchatexporter/" # @param {type:"string"}
# @markdown Google Colab gives you over 100GB of temporary space to work with, which should be way more than enough to store and then download your stuff.
# @markdown
# @markdown  You can also connect your Google Drive instead and copy the stuff to there at the end, though if you just want to download your files immediately without using Google Drive, skip the optional step.
import requests
import json
import os
import re

headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}
if not os.path.exists(SAVE_DIRECTORY):
  os.makedirs(SAVE_DIRECTORY)
SERVER_URL = f'https://www.guilded.gg/api/v1/servers/{SERVER_ID}'
CHANNEL_MSG = f'https://www.guilded.gg/api/v1/channels/{CHANNEL_ID}/messages'
print("Done, do next step")

In [ ]:
# @title (Optional) Connect Google Drive
save_to_gdrive = False # @param {type:"boolean"}
from google.colab import drive
drive.mount('/content/drive')
if save_to_gdrive:
    SAVE_DIRECTORY = "/content/drive/MyDrive/guildedchatexporter/"
print("Done, do next step")

In [ ]:
# @title Get Server status and Members for the chat
memberlist = requests.get(SERVER_URL+"/members", headers=headers)
memberlist.raise_for_status()
#print(json.dumps(memberlist.json(), indent=4))
print("Done, do next step")



In [ ]:
# @title Get Messages
def get_channel_messages(channel_id, after=None, limit=100):
    params = {
        'limit': limit,
        'after': after
    }
    url = f'https://www.guilded.gg/api/v1/channels/{channel_id}/messages'
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

def fetch_all_messages(channel_id):
    all_messages = []
    after = "2015-01-01T00:00:00.000Z"

    while True:
        data = get_channel_messages(channel_id, after=after, limit=100)
        messages = data.get('messages', [])
        all_messages.extend(messages)

        if len(messages) < 100:
            break

        after = messages[-1]['createdAt']

    return all_messages

all_messages = fetch_all_messages(CHANNEL_ID)

CHANNEL_URL = f'https://www.guilded.gg/api/v1/channels/{CHANNEL_ID}'
channelinfo = requests.get(CHANNEL_URL, headers=headers)
channelinfo.raise_for_status()
print(json.dumps(channelinfo.json(), indent=4))
channelname = channelinfo.json()['channel']['name']
channeltopic = channelinfo.json()['channel'].get('topic', '') # In case no topic is set
# Figure out Usernames so we don't have gibberish
members = memberlist.json()["members"]


print(f"Found {len(all_messages)} messages.")


In [ ]:
# @title (Optional) Print all messages and save raw results
Print_JSON = False # @param {type:"boolean"}
Save_JSON = True # @param {type:"boolean"}
if Print_JSON:
    print(json.dumps(all_messages, indent=4))
if Save_JSON:
    with open(f'{SAVE_DIRECTORY}{CHANNEL_ID}.json', 'w', encoding='utf-8') as file:
        json.dump(all_messages, file, indent=4)
        print(f"JSON of all messages saved to {SAVE_DIRECTORY}{CHANNEL_ID}.json")

In [ ]:
# @title (Optional) Save mildly-formatted .txt chat log

with open(f'{SAVE_DIRECTORY}{CHANNEL_ID}.txt', 'w', encoding='utf-8') as file:
    for message in all_messages:
        created_by = message['createdBy']
        created_at = message['createdAt']
        content = message['content']
        username = next((m['user']['name'] for m in members if m['user']['id'] == created_by), "System Message")
        formatted_message = f"{username} ({created_by}) ({created_at}): {content}\n"

        file.write(formatted_message)

print("Chat log saved")

In [ ]:
# @title HTML File Setup
# @markdown Has the base HTML and CSS information which you can edit. Just click RUN to set the info for the next step.
# @markdown
# @markdown This will have a flag in the future for if you don't want to download/show attachments, but why?
## HTML Template

CSS = """
@font-face {
  font-family: "Builder Sans";
  src: url("https://www.guilded.gg/fonts/BuilderSans-Regular.woff2") format('woff2');
}

body {
  font-family: 'Builder Sans', Tahoma, sans-serif;
  color: white;
  background-color: #373943;
}

#chat-log {
  background-color: #373943;
  padding: 16px;
  display: flex;
  flex-direction: column;
  gap: 8px;
}

.message {
  color: #ececee;
  font-size: 15px;
  line-height: 145%;
  font-weight: normal;
  font-family: 'Builder Sans', Tahoma, sans-serif;
  margin-top: 0px;
  min-height: 44px;
  padding: 8px 12px;
  display: flex;
  align-content: flex-start;
  gap: 8px;
  transition: background-color 140ms ease;
  position: relative;
  contain: layout;
  max-width: 80%;
}

.message:hover {
  background-color: #31333c;
}

.created-at {
  margin-right: 8px;
  height: 11px;
  color: #a3a3ac;
  font-size: 13px;
  line-height: 120%;
  font-weight: normal;
  white-space: nowrap;
  pointer-events: auto;
}

.created-by {
  font-size: 15px;
  line-height: 145%;
  font-weight: 700;
  white-space: nowrap;
  pointer-events: auto;
}

.pfp {
  width: 40px;
  height: 40px;
  border-radius: 50%;
  overflow: hidden;
  vertical-align: middle;
  flex-shrink: 0;
  margin-right: 12px;
  pointer-events: auto;
}

.content {
  margin: 0;
  word-break: break-all;
}

.content img,
.content video {
  max-height: 320px;
  max-width: 360px;
  object-fit: contain;
}

"""

HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>#{title}</title>
    <style>
        {CSS}
    </style>
</head>
<body>
    <div id="chat-log">
        <p>#{title}</p>
        <p>{topic}</p>
        <hr>
        {messages}
    </div>
</body>
</html>
"""

## Message Template
MESSAGE_TEMPLATE = """
<div class="message">
    <img class="pfp" src="{avatar_url}">
    <div>
        <span class="created-by">{created_by}</span>
        <span class="created-at">({created_at})</span>
        <p class="content">{content}</p>
    </div>
</div>
"""

print("HTML ready to print")


In [ ]:
# @title Parse and save results into HTML file!

from urllib.parse import urlparse, unquote
import random

formatted_messages = []

for message in all_messages:
    created_by = message['createdBy']
    created_at = message['createdAt']
    content = message['content']

    # Find inline attachments and replace them with relative links or embedded content
    inline_attachments = re.findall(r'!\[\]\((.*?)\)', content)
    for attachment in inline_attachments:
        parsed_url = urlparse(attachment)
        filename = os.path.basename(unquote(parsed_url.path))
        relative_path = f'{CHANNEL_ID}/attachments/{filename}'

        if filename.endswith('.webp'):
            content = content.replace(f'![]({attachment})', f'<a href="{relative_path}"><img src="{relative_path}" alt="{filename}"></a>')
        elif filename.endswith('.mp4'):
            content = content.replace(f'![]({attachment})', f'<video controls><source src="{relative_path}" type="video/webm">Your browser does not support the video tag.</video>')
        else:
            content = content.replace(f'![]({attachment})', f'<a href="{attachment}" target="_blank">{filename}(attachment)</a>')


    username = next((m['user']['name'] for m in members if m['user']['id'] == created_by), "System Message")
    avatar_url = next((m['user']['avatar'] for m in members if m['user']['id'] == created_by), "https://www.guilded.gg/asset/DefaultUserAvatars/profile_5.png")
    avatar_filename = os.path.basename(unquote(urlparse(avatar_url).path))
    avatar_url = f"{CHANNEL_ID}/avatars/{avatar_filename}"

    formatted_message = MESSAGE_TEMPLATE.format(
        avatar_url=avatar_url,
        created_by=username,
        created_at=created_at,
        content=content
    )
    formatted_messages.append(formatted_message)


html_content = HTML_TEMPLATE.format(CSS=CSS, title=channelname, topic=channeltopic, messages='\n'.join(formatted_messages))

with open(os.path.join(SAVE_DIRECTORY, f'{CHANNEL_ID}.html'), 'w', encoding='utf-8') as file:
    file.write(html_content)

print(f"HTML file saved! Get it at {SAVE_DIRECTORY}{CHANNEL_ID}.html! Run the next step to get your attachments.")

In [ ]:
# @title Download all images/videos & member avatars
# @markdown WARNING: Does not save file upload attachments that are not images or videos. Workaround might have to end up self-botting as Bot API simply can't interact with all other uploads.
# @markdown
# @markdown todo currently downloads all member pfps but we should not do that


# Create the attachments folder if it doesn't exist
attachments_dir = os.path.join(SAVE_DIRECTORY, CHANNEL_ID, 'attachments', )
os.makedirs(attachments_dir, exist_ok=True)

def download_attachments(messages, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    url_pattern = r'!\[(.*?)\]\((.*?)\)'

    for message in messages:
        content = message['content']
        matches = re.findall(url_pattern, content)

        for match in matches:
            url = match[1]
            parsed_url = urlparse(url)
            clean_filename = os.path.basename(unquote(parsed_url.path))
            filename = os.path.join(output_dir, clean_filename)

            try:
                response = requests.get(url)
                response.raise_for_status()

                with open(filename, 'wb') as file:
                    file.write(response.content)

                print(f"Downloaded attachment from {url} to {filename}")
            except requests.exceptions.RequestException as e:
                print(f"Error downloading attachment from {url}: {e}")

# Avatars
avatars_dir = os.path.join(SAVE_DIRECTORY, CHANNEL_ID, 'avatars', )
os.makedirs(avatars_dir, exist_ok=True)

def download_avatars(members, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for member in members:
        user = member['user']
        if 'avatar' in user:
            avatar_url = user['avatar']

            if not avatar_url:
                continue

            parsed_url = urlparse(avatar_url)
            clean_filename = os.path.basename(unquote(parsed_url.path))
            filename = os.path.join(output_dir, clean_filename)

            try:
                response = requests.get(avatar_url)
                response.raise_for_status()

                with open(filename, 'wb') as file:
                    file.write(response.content)

                print(f"Downloaded avatar from {avatar_url} to {filename}")
            except requests.exceptions.RequestException as e:
                print(f"Error downloading avatar from {avatar_url}: {e}")

all_messages = fetch_all_messages(CHANNEL_ID)
output_dir = f'{SAVE_DIRECTORY}{CHANNEL_ID}'+'/attachments'
download_attachments(all_messages, output_dir)
download_avatars(members, avatars_dir)
print("All done!")

In [ ]:
# @title Pack all outputted files into ZIP and download! (And save to GDrive if enabled)

%cd {SAVE_DIRECTORY}
!zip -r {CHANNEL_ID}.zip {CHANNEL_ID}.json {CHANNEL_ID}.txt {CHANNEL_ID}.html {CHANNEL_ID}
from google.colab import files
files.download(f"{CHANNEL_ID}.zip")
print("Hot and ready!")

try: # skip gdrive if user skipped it
    if save_to_gdrive:
        !cp {CHANNEL_ID}.zip /content/drive/MyDrive/guildedchatexporter/
        print(f"Copied {CHANNEL_ID}.zip to /content/drive/MyDrive/guildedchatexporter/")
except NameError:
    pass


In [ ]:
# @title save forum channel entries to individual txt files
# @markdown make sure you did "Get Server status and Members for the chat" first
forum_url = f"https://www.guilded.gg/api/v1/channels/{CHANNEL_ID}/topics"
response = requests.get(forum_url, headers=headers, params={'limit': 100} )
response.raise_for_status()
#print(json.dumps(response.json(), indent=4))

for forumTopic in response.json()['forumTopics']:
  forumTopicId = forumTopic['id']
  comments_url = f'https://www.guilded.gg/api/v1/channels/{CHANNEL_ID}/topics/{forumTopicId}/comments'
  comments_response = requests.get(comments_url, headers=headers, params={'limit': 100})
  comments_response.raise_for_status()
  #print(json.dumps(comments_response.json(), indent=4))


members = memberlist.json()["members"]
for forumTopic in response.json()['forumTopics']:
  forumTopicId = forumTopic['id']
  forumtitle = forumTopic['title']
  forumcreated_by = forumTopic['createdBy']
  forumcreated_at = forumTopic['createdAt']

  firsturl = f'https://www.guilded.gg/api/v1/channels/{CHANNEL_ID}/topics/{forumTopicId}'
  firstresp = requests.get(firsturl, headers=headers)


  comments_url = f'https://www.guilded.gg/api/v1/channels/{CHANNEL_ID}/topics/{forumTopicId}/comments'
  comments_response = requests.get(comments_url, headers=headers, params={'limit': 100})

  #print(json.dumps(comments_response.json(), indent=4))
  txttitle = re.sub(r'[<>:"/\\|?*]', '_', forumtitle)
  os.makedirs(os.path.dirname(f'{SAVE_DIRECTORY}forum/{CHANNEL_ID}/{txttitle}.txt'), exist_ok=True)
  with open(f'{SAVE_DIRECTORY}forum/{CHANNEL_ID}/{txttitle}.txt', 'w', encoding='utf-8') as file:
    #write the title, then newline, username, createdby, createdat
    file.write(f"{forumtitle}\n\n")
    #get the right username for the first poster
    username = next((m['user']['name'] for m in members if m['user']['id'] == forumcreated_by), "[deleted user]")
    file.write(f"{username} ({forumcreated_by}) ({forumcreated_at}): {firstresp.json()['forumTopic']['content']}\n")


    for message in comments_response.json()['forumTopicComments']:
        created_by = message['createdBy']
        created_at = message['createdAt']
        content = message['content']
        username = next((m['user']['name'] for m in members if m['user']['id'] == created_by), "[deleted user]")
        formatted_message = f"{username} ({created_by}) ({created_at}): {content}\n"

        file.write(formatted_message)
        # notify when end of comments_response reached

print(f"done saving to /guildedchatexporter/forum/CHANNEL_ID")

